In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, ToolMessage, AIMessage
from langchain_core.messages.human import HumanMessage
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import os

load_dotenv()
tavily_api_key = os.environ.get("TAVILY_API_KEY")


In [2]:
tool = TavilySearchResults(
    max_results=2
) # 2 results per query
print (type(tool))
print (tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [3]:
# AgentState 类定义了代理的状态数据结构
# 这是一个 TypedDict 类型的类,用于定义具有类型提示的字典
# 包含一个 messages 字段:
# - messages 字段是一个列表,存储 AnyMessage 类型的消息
# - 使用 Annotated 和 operator.add 标注,表示该字段支持列表拼接操作
# - AnyMessage 是一个联合类型,可以是各种消息类型(AIMessage、HumanMessage等)
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

我们需要一个函数来调用`Ollama`，一个函数来检查是否存在某种action，以及一个执行action的函数。

In [4]:
# 配置日志记录系统
import logging

# 设置基本的日志配置
# level=logging.INFO 表示将捕获 INFO 级别及以上的所有日志消息
# 日志级别从低到高: DEBUG < INFO < WARNING < ERROR < CRITICAL
logging.basicConfig(level=logging.INFO)

# 创建一个日志记录器实例
# __name__ 是当前模块的名称
# 这个日志记录器将被 Agent 类用来记录执行过程中的信息
logger = logging.getLogger(__name__)
import sys
from datetime import datetime
from pathlib import Path

# 创建logs目录（如果不存在）
log_dir = Path("./logs")
log_dir.mkdir(exist_ok=True)

# 生成日志文件名（包含时间戳）
log_file = log_dir / f"agent_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

# 配置日志格式
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# 文件处理器
file_handler = logging.FileHandler(log_file)
file_handler.setFormatter(formatter)
file_handler.setLevel(logging.INFO)

# 控制台处理器（带颜色）
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(logging.Formatter(
    '%(asctime)s - \033[1;36m%(levelname)s\033[0m - %(message)s'))
console_handler.setLevel(logging.INFO)

# 配置logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(file_handler)
logger.addHandler(console_handler)

In [5]:
class Agent:
    """
    Agent class that orchestrates interaction between a language model and tools.
    Uses a state graph to manage conversation flow and decision making process.
    """

    def __init__(self, model, tools, system=""):
        """
        Initialize the agent with a language model, tools, and optional system message.
        
        Args:
            model: Language model that will generate responses
            tools: List of tools the agent can use
            system: Optional system prompt to guide agent behavior
            
        Creates a state graph with the following workflow:
            1. Start with LLM node
            2. Check if action is needed
            3. Execute action if needed, then return to LLM
            4. End conversation if no action is needed
        """
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
        logger.info(f"model: {self.model}")

    def exists_action(self, state: AgentState):
        """
        Determine if an action needs to be taken based on the last message.
        
        Args:
            state: Current conversation state containing messages
            
        Returns:
            Boolean indicating if the latest AI message contains tool calls
            that need to be executed
        """
        result = state['messages'][-1]
        logger.info(f"exists_action result: {result}")
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        """
        Send the current conversation state to the language model and get a response.
        
        Args:
            state: Current conversation state containing message history
            
        Returns:
            Updated state with the new AI message appended
            
        If a system prompt is provided, it's added at the beginning of messages
        before sending to the model.
        """
        logger.info(f"state: {state}")
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        logger.info(f"LLM message: {message}")
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        """
        Execute tool calls requested by the language model.
        
        Args:
            state: Current conversation state containing message history
            
        Returns:
            Updated state with tool execution results appended as ToolMessages
            
        Process:
        1. Extract tool calls from the last message
        2. For each tool call, validate the tool name and execute it
        3. Format results as ToolMessages with appropriate IDs
        4. Return all results to be processed by the model in the next step
        """
        import threading
        print(f"take_action called in thread: {threading.current_thread().name}")
        tool_calls = state['messages'][-1].tool_calls
        results = []
        print(f"take_action called with tool_calls: {tool_calls}")
        for t in tool_calls:
            logger.info(f"Calling: {t}")
            print(f"Calling: {t}") 
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
                logger.info(f"action {t['name']}, result: {result}")
                print(f"action {t['name']}, result: {result}")
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information.  \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOllama(
    model="qwen2.5:7b",
    temperature=0
)
abot = Agent(model, [tool], system=prompt)

2025-03-11 21:14:02,900 - INFO - model: bound=ChatOllama(model='qwen2.5:7b', temperature=0.0) kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]} config={} config_factories=[]


INFO:__main__:model: bound=ChatOllama(model='qwen2.5:7b', temperature=0.0) kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]} config={} config_factories=[]


In [7]:
messages = [
    HumanMessage(
        content="What is the weather in sf?"
    )
]
result = abot.graph.invoke({'messages': messages})

2025-03-11 21:14:02,906 - INFO - state: {'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={})]}


INFO:__main__:state: {'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={})]}
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


2025-03-11 21:14:06,313 - INFO - LLM message: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:06.304957Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3394111041, 'load_duration': 563187208, 'prompt_eval_count': 244, 'prompt_eval_duration': 1593000000, 'eval_count': 26, 'eval_duration': 1007000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-df7ea652-9a66-49a0-8f3f-b37064aaa390-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '6b465965-9d78-4219-b5b6-8b17cd71f84b', 'type': 'tool_call'}] usage_metadata={'input_tokens': 244, 'output_tokens': 26, 'total_tokens': 270}


INFO:__main__:LLM message: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:06.304957Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3394111041, 'load_duration': 563187208, 'prompt_eval_count': 244, 'prompt_eval_duration': 1593000000, 'eval_count': 26, 'eval_duration': 1007000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-df7ea652-9a66-49a0-8f3f-b37064aaa390-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '6b465965-9d78-4219-b5b6-8b17cd71f84b', 'type': 'tool_call'}] usage_metadata={'input_tokens': 244, 'output_tokens': 26, 'total_tokens': 270}


2025-03-11 21:14:06,316 - INFO - exists_action result: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:06.304957Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3394111041, 'load_duration': 563187208, 'prompt_eval_count': 244, 'prompt_eval_duration': 1593000000, 'eval_count': 26, 'eval_duration': 1007000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-df7ea652-9a66-49a0-8f3f-b37064aaa390-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '6b465965-9d78-4219-b5b6-8b17cd71f84b', 'type': 'tool_call'}] usage_metadata={'input_tokens': 244, 'output_tokens': 26, 'total_tokens': 270}


...

INFO:__main__:state: {'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:06.304957Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3394111041, 'load_duration': 563187208, 'prompt_eval_count': 244, 'prompt_eval_duration': 1593000000, 'eval_count': 26, 'eval_duration': 1007000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-df7ea652-9a66-49a0-8f3f-b37064aaa390-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '6b465965-9d78-4219-b5b6-8b17cd71f84b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 244, 'output_tokens': 26, 'total_tokens': 270}), ToolMessage(content='[{\'title\': \'Weather in San Francisco\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'

2025-03-11 21:14:22,475 - INFO - LLM message: content='The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Coverage: 75%\n\nThe temperature feels like 6.1°C (43°F). Visibility is good at 16 km (9 miles).\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:22.468568Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8480100833, 'load_duration': 35982583, 'prompt_eval_count': 987, 'prompt_eval_duration': 3483000000, 'eval_count': 115, 'eval_duration': 4938000000, 'message': Message(role='assistant', content='The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Coverage: 7

INFO:__main__:LLM message: content='The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Coverage: 75%\n\nThe temperature feels like 6.1°C (43°F). Visibility is good at 16 km (9 miles).\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:22.468568Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8480100833, 'load_duration': 35982583, 'prompt_eval_count': 987, 'prompt_eval_duration': 3483000000, 'eval_count': 115, 'eval_duration': 4938000000, 'message': Message(role='assistant', content='The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Coverage: 75%\n\nThe temperatu

2025-03-11 21:14:22,477 - INFO - exists_action result: content='The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Coverage: 75%\n\nThe temperature feels like 6.1°C (43°F). Visibility is good at 16 km (9 miles).\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:22.468568Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8480100833, 'load_duration': 35982583, 'prompt_eval_count': 987, 'prompt_eval_duration': 3483000000, 'eval_count': 115, 'eval_duration': 4938000000, 'message': Message(role='assistant', content='The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Co

INFO:__main__:exists_action result: content='The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Coverage: 75%\n\nThe temperature feels like 6.1°C (43°F). Visibility is good at 16 km (9 miles).\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:22.468568Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8480100833, 'load_duration': 35982583, 'prompt_eval_count': 987, 'prompt_eval_duration': 3483000000, 'eval_count': 115, 'eval_duration': 4938000000, 'message': Message(role='assistant', content='The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Coverage: 75%\n\nThe 

In [8]:
result['messages'][-1].content

'The current weather in San Francisco is as follows:\n\n- Temperature: 7.2°C (45°F)\n- Condition: Partly cloudy\n- Wind Speed: 4.0 mph (6.5 kph) from the WSW direction\n- Humidity: 71%\n- Cloud Coverage: 75%\n\nThe temperature feels like 6.1°C (43°F). Visibility is good at 16 km (9 miles).\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).'

In [9]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({'messages': messages})

2025-03-11 21:14:22,498 - INFO - state: {'messages': [HumanMessage(content='What is the weather in SF and LA?', additional_kwargs={}, response_metadata={})]}


INFO:__main__:state: {'messages': [HumanMessage(content='What is the weather in SF and LA?', additional_kwargs={}, response_metadata={})]}
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


2025-03-11 21:14:23,880 - INFO - LLM message: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:23.878975Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1377691583, 'load_duration': 13891666, 'prompt_eval_count': 246, 'prompt_eval_duration': 180000000, 'eval_count': 30, 'eval_duration': 1180000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-be5020e8-6a00-4fc3-bcb6-afe9ecc9171d-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco and Los Angeles'}, 'id': '75b27a48-70c4-4987-9338-de0ee7f1db6a', 'type': 'tool_call'}] usage_metadata={'input_tokens': 246, 'output_tokens': 30, 'total_tokens': 276}


INFO:__main__:LLM message: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:23.878975Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1377691583, 'load_duration': 13891666, 'prompt_eval_count': 246, 'prompt_eval_duration': 180000000, 'eval_count': 30, 'eval_duration': 1180000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-be5020e8-6a00-4fc3-bcb6-afe9ecc9171d-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco and Los Angeles'}, 'id': '75b27a48-70c4-4987-9338-de0ee7f1db6a', 'type': 'tool_call'}] usage_metadata={'input_tokens': 246, 'output_tokens': 30, 'total_tokens': 276}


2025-03-11 21:14:23,881 - INFO - exists_action result: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:23.878975Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1377691583, 'load_duration': 13891666, 'prompt_eval_count': 246, 'prompt_eval_duration': 180000000, 'eval_count': 30, 'eval_duration': 1180000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-be5020e8-6a00-4fc3-bcb6-afe9ecc9171d-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco and Los Angeles'}, 'id': '75b27a48-70c4-4987-9338-de0ee7f1db6a', 'type': 'tool_call'}] usage_metadata={'input_tokens': 246, 'output_tokens': 30, 'total_tokens': 276}


...

INFO:__main__:state: {'messages': [HumanMessage(content='What is the weather in SF and LA?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:23.878975Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1377691583, 'load_duration': 13891666, 'prompt_eval_count': 246, 'prompt_eval_duration': 180000000, 'eval_count': 30, 'eval_duration': 1180000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-be5020e8-6a00-4fc3-bcb6-afe9ecc9171d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco and Los Angeles'}, 'id': '75b27a48-70c4-4987-9338-de0ee7f1db6a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 246, 'output_tokens': 30, 'total_tokens': 276}), ToolMessage(content='[{\'title\': \'Weather in San Francisco, Los Angeles\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\

2025-03-11 21:14:41,755 - INFO - LLM message: content="I couldn't find the exact weather information for San Francisco (SF) and Los Angeles (LA). However, I found some related content:\n\n- [Weather API](https://www.weatherapi.com/) provided data for a location with coordinates near Colonia Los Angeles in Honduras. The temperature there is 19.3°C or 66.7°F with light rain.\n\n- For San Francisco, CA, the weather forecast shows a chance of rain throughout the night with temperatures ranging from 46°F to 49°F.\n\nYou might want to check these links for more accurate and up-to-date information on the specific locations you're interested in." additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:41.753345Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9427892292, 'load_duration': 41852209, 'prompt_eval_count': 1015, 'prompt_eval_duration': 3612000000, 'eval_count': 133, 'eval_duration': 5751000000, 'message': Message(role='assistant', con

INFO:__main__:LLM message: content="I couldn't find the exact weather information for San Francisco (SF) and Los Angeles (LA). However, I found some related content:\n\n- [Weather API](https://www.weatherapi.com/) provided data for a location with coordinates near Colonia Los Angeles in Honduras. The temperature there is 19.3°C or 66.7°F with light rain.\n\n- For San Francisco, CA, the weather forecast shows a chance of rain throughout the night with temperatures ranging from 46°F to 49°F.\n\nYou might want to check these links for more accurate and up-to-date information on the specific locations you're interested in." additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:41.753345Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9427892292, 'load_duration': 41852209, 'prompt_eval_count': 1015, 'prompt_eval_duration': 3612000000, 'eval_count': 133, 'eval_duration': 5751000000, 'message': Message(role='assistant', content="I couldn't fi

2025-03-11 21:14:41,756 - INFO - exists_action result: content="I couldn't find the exact weather information for San Francisco (SF) and Los Angeles (LA). However, I found some related content:\n\n- [Weather API](https://www.weatherapi.com/) provided data for a location with coordinates near Colonia Los Angeles in Honduras. The temperature there is 19.3°C or 66.7°F with light rain.\n\n- For San Francisco, CA, the weather forecast shows a chance of rain throughout the night with temperatures ranging from 46°F to 49°F.\n\nYou might want to check these links for more accurate and up-to-date information on the specific locations you're interested in." additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:41.753345Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9427892292, 'load_duration': 41852209, 'prompt_eval_count': 1015, 'prompt_eval_duration': 3612000000, 'eval_count': 133, 'eval_duration': 5751000000, 'message': Message(role='assist

INFO:__main__:exists_action result: content="I couldn't find the exact weather information for San Francisco (SF) and Los Angeles (LA). However, I found some related content:\n\n- [Weather API](https://www.weatherapi.com/) provided data for a location with coordinates near Colonia Los Angeles in Honduras. The temperature there is 19.3°C or 66.7°F with light rain.\n\n- For San Francisco, CA, the weather forecast shows a chance of rain throughout the night with temperatures ranging from 46°F to 49°F.\n\nYou might want to check these links for more accurate and up-to-date information on the specific locations you're interested in." additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:14:41.753345Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9427892292, 'load_duration': 41852209, 'prompt_eval_count': 1015, 'prompt_eval_duration': 3612000000, 'eval_count': 133, 'eval_duration': 5751000000, 'message': Message(role='assistant', content="I co

In [10]:
result['messages'][-1].content

"I couldn't find the exact weather information for San Francisco (SF) and Los Angeles (LA). However, I found some related content:\n\n- [Weather API](https://www.weatherapi.com/) provided data for a location with coordinates near Colonia Los Angeles in Honduras. The temperature there is 19.3°C or 66.7°F with light rain.\n\n- For San Francisco, CA, the weather forecast shows a chance of rain throughout the night with temperatures ranging from 46°F to 49°F.\n\nYou might want to check these links for more accurate and up-to-date information on the specific locations you're interested in."

In [11]:
query = "Who won the super bowl in 2024? What is the GDP of that state?"
messages = [HumanMessage(content=query)]
result = abot.graph.invoke({'messages': messages})

2025-03-11 21:35:20,133 - INFO - state: {'messages': [HumanMessage(content='Who won the super bowl in 2024? What is the GDP of that state?', additional_kwargs={}, response_metadata={})]}


INFO:__main__:state: {'messages': [HumanMessage(content='Who won the super bowl in 2024? What is the GDP of that state?', additional_kwargs={}, response_metadata={})]}
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


2025-03-11 21:35:25,681 - INFO - LLM message: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:35:25.680433Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5538585250, 'load_duration': 562959292, 'prompt_eval_count': 257, 'prompt_eval_duration': 2712000000, 'eval_count': 49, 'eval_duration': 2043000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-b229a2b3-c941-440d-a566-d92e885fa854-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': '80e49c63-042e-4386-9d87-31bb992181db', 'type': 'tool_call'}] usage_metadata={'input_tokens': 257, 'output_tokens': 49, 'total_tokens': 306}


INFO:__main__:LLM message: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:35:25.680433Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5538585250, 'load_duration': 562959292, 'prompt_eval_count': 257, 'prompt_eval_duration': 2712000000, 'eval_count': 49, 'eval_duration': 2043000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-b229a2b3-c941-440d-a566-d92e885fa854-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': '80e49c63-042e-4386-9d87-31bb992181db', 'type': 'tool_call'}] usage_metadata={'input_tokens': 257, 'output_tokens': 49, 'total_tokens': 306}


2025-03-11 21:35:25,682 - INFO - exists_action result: content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:35:25.680433Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5538585250, 'load_duration': 562959292, 'prompt_eval_count': 257, 'prompt_eval_duration': 2712000000, 'eval_count': 49, 'eval_duration': 2043000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-b229a2b3-c941-440d-a566-d92e885fa854-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': '80e49c63-042e-4386-9d87-31bb992181db', 'type': 'tool_call'}] usage_metadata={'input_tokens': 257, 'output_tokens': 49, 'total_tokens': 306}


...

INFO:__main__:state: {'messages': [HumanMessage(content='Who won the super bowl in 2024? What is the GDP of that state?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:35:25.680433Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5538585250, 'load_duration': 562959292, 'prompt_eval_count': 257, 'prompt_eval_duration': 2712000000, 'eval_count': 49, 'eval_duration': 2043000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-b229a2b3-c941-440d-a566-d92e885fa854-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': '80e49c63-042e-4386-9d87-31bb992181db', 'type': 'tool_call'}], usage_metadata={'input_tokens': 257, 'output_tokens': 49, 'total_tokens': 306}), ToolMessage(content='[{\'title\': \'Who won the 2024 Super Bowl? Looking back at the overtime thriller\', \'url\': \'https

2025-03-11 21:35:53,512 - INFO - LLM message: content='The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:35:53.506654Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5498215334, 'load_duration': 35426084, 'prompt_eval_count': 1256, 'prompt_eval_duration': 2570000000, 'eval_count': 64, 'eval_duration': 2852000000, 'message': Message(role='assistant', content='The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.', images=None, tool_calls=None)} id='run-8fa2c959-adac-4a23-8808-1dca846bc66b-0' usage_metadata={'input_tokens': 1256, 'output_t

INFO:__main__:LLM message: content='The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:35:53.506654Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5498215334, 'load_duration': 35426084, 'prompt_eval_count': 1256, 'prompt_eval_duration': 2570000000, 'eval_count': 64, 'eval_duration': 2852000000, 'message': Message(role='assistant', content='The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.', images=None, tool_calls=None)} id='run-8fa2c959-adac-4a23-8808-1dca846bc66b-0' usage_metadata={'input_tokens': 1256, 'output_tokens': 64, 'total_

2025-03-11 21:35:53,513 - INFO - exists_action result: content='The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:35:53.506654Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5498215334, 'load_duration': 35426084, 'prompt_eval_count': 1256, 'prompt_eval_duration': 2570000000, 'eval_count': 64, 'eval_duration': 2852000000, 'message': Message(role='assistant', content='The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.', images=None, tool_calls=None)} id='run-8fa2c959-adac-4a23-8808-1dca846bc66b-0' usage_metadata={'input_tokens': 1256, 

INFO:__main__:exists_action result: content='The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-11T13:35:53.506654Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5498215334, 'load_duration': 35426084, 'prompt_eval_count': 1256, 'prompt_eval_duration': 2570000000, 'eval_count': 64, 'eval_duration': 2852000000, 'message': Message(role='assistant', content='The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.', images=None, tool_calls=None)} id='run-8fa2c959-adac-4a23-8808-1dca846bc66b-0' usage_metadata={'input_tokens': 1256, 'output_tokens': 64

In [12]:
result['messages'][-1].content

'The Kansas City Chiefs won the Super Bowl in 2024 (Super Bowl L VIII). As of 2023, the gross domestic product (GDP) of Missouri was approximately $348.5 billion. This figure is inflation-adjusted and based on data from USAFacts.'